In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

/home/lincong/miniconda3/envs/gpt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import gc
import torch

In [3]:
model_path = '/cognitive_comp/lincong/models/Llama-2-13B-Chat-fp16'
tokenizer_path = '/cognitive_comp/lincong/models/Llama-2-13B-Chat-fp16'

In [4]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
tokenizer.pad_token_id = 2

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_path)

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:37<00:00, 12.49s/it]


In [6]:
model = model.cuda()

In [8]:
gen_args = {
    'do_sample': True,
    'top_p': 0.85,
    'top_k': 0,
    'repetition_penalty': 1.0,
    'temperature': 0.85,
    'use_cache': True,
    'max_new_tokens': 2048,
}

In [47]:
data = {"train_query": "Which flower does Grace more likely prefer, rose, tulip, daisy, or lily?", "train_options": ["The answer is dog", "The answer is bird", "The answer is lion", "The answer is elephant", "The answer is cat", "I don't know"], "train_answer": 2, "test_query": "Given the following options of animal, which will probabily attract Grace: dog, lion, elephant, or bird?", "test_options": ["The answer is dog", "The answer is bird", "The answer is lion", "The answer is elephant", "The answer is cat", "I don't know"], "test_answer": 2, "basic": "Grace is a 30-year-old female.", "details": "Her favorite color is red and favortie animal is lion. She enjoys the taste of grape, likes decorating her room with tulip, and listening to hip hop music."}

In [50]:
prompt = ' '.join(['[INST]', data['basic'], data['details'], data['train_query'], '[/INST]'])

In [51]:
input_ids = tokenizer(prompt, return_tensors='pt').input_ids
# print(tokenizer.batch_decode(input_ids, skip_special_token=False)[0])
outputs = model.generate(input_ids.cuda(), **gen_args)
print(tokenizer.decode(outputs[0], skip_special_token=False))

<s> [INST] Grace is a 30-year-old female. Her favorite color is red and favortie animal is lion. She enjoys the taste of grape, likes decorating her room with tulip, and listening to hip hop music. Which flower does Grace more likely prefer, rose, tulip, daisy, or lily? [/INST]  Based on the information provided, it is more likely that Grace prefers tulips. This is because:

1. Her favorite color is red, and tulips come in a variety of red shades.
2. She enjoys decorating her room with tulips, which suggests that she has a strong affinity for the flower.
3. The description of her taste in music, hip hop, is often associated with bold and vibrant colors, like red, which is also a characteristic of tulips.

Therefore, it is less likely that Grace prefers daisy, lily, or rose, as these flowers are not as closely aligned with her preferences in color and style.</s>


In [20]:
input_text = """[INST] <<SYS>>
You are a hard-working student who is refining your answer to an assignment question following your teacher's advice.
<</SYS>>

You have submitted an answer to your assignment question, and your teacher gives you advice to improve your answer:

[QUESTION]: Do we spend only a small part of the US Federal budget on foreign aid to poor countries?

[YOUR ANSWER]: No, the amount of the US Federal budget spent on foreign aid to poor countries is not a small part. While it is true that foreign aid is a relatively small percentage of the overall federal budget, it is still a significant amount of money.

According to the Office of Management and Budget (OMB), in fiscal year 2020, the US government spent approximately $40.5 billion on foreign aid, which is about 1.4% of the total federal budget. This includes funding for the State Department, the U.S. Agency for International Development (USAID), and other programs that support international development and humanitarian assistance.

To put this amount into perspective, $40.5 billion is more than the annual GDP of many countries around the world. It is also more than the total amount of money raised by all Kickstarter campaigns in 2020.

While $40.5 billion may seem like a lot of money, it is important to remember that it is a small fraction of the total federal budget. In fiscal year 2020, the federal government spent over $4 trillion, so foreign aid accounts for only about 1% of the total federal spending.

However, it is worth noting that foreign aid is a critical investment in the global community and can have a significant impact on poverty reduction, economic development, and humanitarian assistance. The money spent on foreign aid can help improve healthcare, education, and infrastructure in developing countries, and can also support humanitarian responses to natural disasters and conflicts.

In conclusion, while foreign aid is not a large percentage of the overall federal budget, it is still a significant amount of money that can have a meaningful impact on the global community. It is important to continue investing in foreign aid to support the world's most vulnerable populations and promote economic development and stability.

[TEACHER ADVICE]: The response, while informative, could have been more effective in addressing the user\'s query. The user asked if foreign aid is a small part of the US Federal budget, but the response does not directly answer this question. Instead, it provides information about the percentage of the total federal budget spent on foreign aid, which is not directly relevant to the user\'s query. \n\nThe response could have been clearer in its structure and presentation of information. The use of headings, bullet points, or other formatting devices could have improved the readability of the response. For instance, the breakdown of the $40.5 billion figure into its various components, such as the amounts spent on different programs, could have been presented in a clearer manner.\n\nThe response could have been more concise. The user\'s query was a simple question, but the response included a lot of additional information that was not directly relevant to the question. For example, the comparison of the amount of money spent on foreign aid to the GDP of other countries or the total amount of money raised by all Kickstarter campaigns in 2020 is not directly relevant to the user\'s query.\n\nThe response could have cited official documents, links, references, statistics, research or other credible sources to support its claim. For instance, it could have cited the OMB\'s official data on the percentage of the federal budget spent on foreign aid.\n\nThe response could have been more balanced by considering different perspectives. For instance, it could have mentioned that some people might argue that the amount of money spent on foreign aid is too high and could be better spent on domestic issues.\n\nThe response could have been more engaging by asking a rhetorical question or using a more conversational tone. For example, it could have started with a question like "Is it surprising that the US spends a significant amount of its federal budget on foreign aid?"\n\nRating: [[6]]

Write a new answer to the question. You should learn from your teacher's advice to avoid mistakes in your previous answer and make improvement. Please write only the answer and don't repeat the question or include how you follow the advice. [YOUR ANSWER]: [/INST]"""
input_ids = tokenizer(input_text, return_tensors='pt').input_ids
# print(tokenizer.batch_decode(input_ids, skip_special_token=False)[0])
outputs = model.generate(input_ids.cuda(), **gen_args)
print(tokenizer.decode(outputs[0], skip_special_token=False))

<s> [INST] <<SYS>>
You are a hard-working student who is refining your answer to an assignment question following your teacher's advice.
<</SYS>>

You have submitted an answer to your assignment question, and your teacher gives you advice to improve your answer:

[QUESTION]: Do we spend only a small part of the US Federal budget on foreign aid to poor countries?

[YOUR ANSWER]: No, the amount of the US Federal budget spent on foreign aid to poor countries is not a small part. While it is true that foreign aid is a relatively small percentage of the overall federal budget, it is still a significant amount of money.

According to the Office of Management and Budget (OMB), in fiscal year 2020, the US government spent approximately $40.5 billion on foreign aid, which is about 1.4% of the total federal budget. This includes funding for the State Department, the U.S. Agency for International Development (USAID), and other programs that support international development and humanitarian assis

In [16]:
input_text = """[INST] Imagine you are a student who is refining your assignment according to your teacher's comment on your previous work. 

This is the question in your assignment: Do we spend only a small part of the US Federal budget on foreign aid to poor countries?

This is your previous answer: No, the amount of the US Federal budget spent on foreign aid to poor countries is not a small part. While it is true that foreign aid is a relatively small percentage of the overall federal budget, it is still a significant amount of money.

According to the Office of Management and Budget (OMB), in fiscal year 2020, the US government spent approximately $40.5 billion on foreign aid, which is about 1.4% of the total federal budget. This includes funding for the State Department, the U.S. Agency for International Development (USAID), and other programs that support international development and humanitarian assistance.

To put this amount into perspective, $40.5 billion is more than the annual GDP of many countries around the world. It is also more than the total amount of money raised by all Kickstarter campaigns in 2020.

While $40.5 billion may seem like a lot of money, it is important to remember that it is a small fraction of the total federal budget. In fiscal year 2020, the federal government spent over $4 trillion, so foreign aid accounts for only about 1% of the total federal spending.

However, it is worth noting that foreign aid is a critical investment in the global community and can have a significant impact on poverty reduction, economic development, and humanitarian assistance. The money spent on foreign aid can help improve healthcare, education, and infrastructure in developing countries, and can also support humanitarian responses to natural disasters and conflicts.

In conclusion, while foreign aid is not a large percentage of the overall federal budget, it is still a significant amount of money that can have a meaningful impact on the global community. It is important to continue investing in foreign aid to support the world's most vulnerable populations and promote economic development and stability.

And your teacher's comment is: The response, while informative, could have been more effective in addressing the user\'s query. The user asked if foreign aid is a small part of the US Federal budget, but the response does not directly answer this question. Instead, it provides information about the percentage of the total federal budget spent on foreign aid, which is not directly relevant to the user\'s query. \n\nThe response could have been clearer in its structure and presentation of information. The use of headings, bullet points, or other formatting devices could have improved the readability of the response. For instance, the breakdown of the $40.5 billion figure into its various components, such as the amounts spent on different programs, could have been presented in a clearer manner.\n\nThe response could have been more concise. The user\'s query was a simple question, but the response included a lot of additional information that was not directly relevant to the question. For example, the comparison of the amount of money spent on foreign aid to the GDP of other countries or the total amount of money raised by all Kickstarter campaigns in 2020 is not directly relevant to the user\'s query.\n\nThe response could have cited official documents, links, references, statistics, research or other credible sources to support its claim. For instance, it could have cited the OMB\'s official data on the percentage of the federal budget spent on foreign aid.\n\nThe response could have been more balanced by considering different perspectives. For instance, it could have mentioned that some people might argue that the amount of money spent on foreign aid is too high and could be better spent on domestic issues.\n\nThe response could have been more engaging by asking a rhetorical question or using a more conversational tone. For example, it could have started with a question like "Is it surprising that the US spends a significant amount of its federal budget on foreign aid?"\n\nRating: [[6]]

Please refine your answer to the question. [/INST]"""
input_ids = tokenizer(input_text, return_tensors='pt').input_ids
# print(tokenizer.batch_decode(input_ids, skip_special_token=False)[0])
outputs = model.generate(input_ids.cuda(), **gen_args)
print(tokenizer.decode(outputs[0], skip_special_token=False))

<s> [INST] Imagine you are a student who is refining your assignment according to your teacher's comment on your previous work. 

This is the question in your assignment: Do we spend only a small part of the US Federal budget on foreign aid to poor countries?

This is your previous answer: No, the amount of the US Federal budget spent on foreign aid to poor countries is not a small part. While it is true that foreign aid is a relatively small percentage of the overall federal budget, it is still a significant amount of money.

According to the Office of Management and Budget (OMB), in fiscal year 2020, the US government spent approximately $40.5 billion on foreign aid, which is about 1.4% of the total federal budget. This includes funding for the State Department, the U.S. Agency for International Development (USAID), and other programs that support international development and humanitarian assistance.

To put this amount into perspective, $40.5 billion is more than the annual GDP of 